In [1]:
import pandas as pd
import os
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.4f}'.format
from functools import reduce
pd.options.display.float_format = '{:.4f}'.format
import seaborn as sns

In [5]:
target_trained_with_nums = [200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000]
dirnames = sorted(glob(os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', '*_seed2')))
dirnames

['C:\\DATA\\ComBERT\\results_spamFiltering\\Araci_CM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\Araci_NoPT_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\Araci_SM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\BERT_CM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\BERT_NoPT_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\BERT_SM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\SECBERT_CM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\SECBERT_NoPT_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\SECBERT_SM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\Yang_CM_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\Yang_NoPT_epoch5_seed2',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\Yang_SM_epoch5_seed2']

In [6]:
dfs = []
for dirname in dirnames:
    model_name, method_name, epoch_num, seed_num = os.path.basename(dirname).split('_')
    
    records = []
    for target_trained_with_num in target_trained_with_nums:
        filepaths = glob(os.path.join(os.path.dirname(dirname), '{}_*'.format('_'.join([model_name, method_name, epoch_num])), \
                  'classification_report_train_{}.csv'.format(target_trained_with_num)))
        
        column_name = '_'.join([model_name, method_name])
        averaged_classification_report_df = get_average_classification_report([pd.read_csv(filepath) for filepath in filepaths])
        acc = 100*averaged_classification_report_df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]
        records.append((target_trained_with_num, acc))
    dfs.append(pd.DataFrame(records, columns=['trained_with', column_name]).sort_values(by=['trained_with']))

result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
result.set_index('trained_with', inplace=True)
result.loc['Average'] = result.mean()

result

,Araci_CM,Araci_NoPT,Araci_SM,BERT_CM,BERT_NoPT,BERT_SM,SECBERT_CM,SECBERT_NoPT,SECBERT_SM,Yang_CM,Yang_NoPT,Yang_SM
trained_with,,,,,,,,,,,,
200,72.5800,64.7100,75.8267,71.8600,75.7767,75.4667,76.3833,73.4333,72.6167,72.7667,72.8433,75.1100
300,74.2133,71.2700,78.9533,77.5733,76.4567,78.2300,80.1800,77.9033,75.8967,78.9667,77.5167,79.4633
400,77.5400,70.7367,79.9500,79.0567,74.0833,77.3367,80.7600,77.5300,76.0700,76.8667,79.9067,79.5333
500,79.5133,73.0867,79.8633,78.7433,78.5400,79.0700,80.5933,79.1100,78.2967,79.2200,79.7667,80.2767
600,79.8833,73.7800,80.6633,79.4200,78.6433,79.3900,80.2600,80.2367,79.1600,80.1133,77.0600,80.0067
700,79.1933,75.1633,80.0200,78.9467,79.3767,79.3633,80.5133,80.2933,78.9000,80.2100,79.1867,79.8133
800,79.0033,76.8500,78.2700,78.4533,78.8200,78.5833,79.0133,79.2433,77.7667,78.9633,79.2833,78.9533
900,80.1700,78.3467,79.8333,78.1300,80.0767,80.7033,79.3333,79.0033,80.3933,80.8167,80.0400,78.4533
1000,79.9700,77.4400,79.8233,80.3233,81.0067,79.5267,79.8300,79.6833,78.6367,79.4833,74.9600,79.3100


Average of the four models

In [7]:
dfs = []
for method_name in sorted(set([item.split('_')[-1] for item in result.columns])):
    colname = method_name
    dfs.append(pd.DataFrame(result[[item for item in result.columns if colname in item]].mean(axis=1), columns=[colname]))
average_result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
average_result

,CM,NoPT,SM
trained_with,,,
200,73.3975,71.6908,74.7550
300,77.7333,75.7867,78.1358
400,78.5558,75.5642,78.2225
500,79.5175,77.6258,79.3767
600,79.9192,77.4300,79.8050
700,79.7158,78.5050,79.5242
800,78.8583,78.5492,78.3933
900,79.6125,79.3667,79.8458
1000,79.9017,78.2725,79.3242
